# 1
### МЛ модель використовує фрейми з веб-камери на комп’ютері, розпізнає лице людини і обводить його прямокутником. Напиши скріпт який автоматично виведе це на екран. Виконай це завдання найзручнішим та найшвидшим шляхом. 


Порівняймо моделі багатозадачної каскадної згорткової нейронної мережі (MTCNN) та модуль DNN OpenCV.

Модель Caffe базується на Single Shot-Multibox Detector (SSD) і використовує архітектуру ResNet-10 як свою основу (був представлений після OpenCV 3.3 у своєму модулі глибокої нейронної мережі).
MTCNN був представлений Kaipeng Zhang та ін. у 2016 р. у своїй роботі « Спільне виявлення обличчя та вирівнювання за допомогою багатозадачних каскадних згорткових мереж ». У ньому використовується каскадна структура з трьома ступенями CNN

In [5]:
import cv2
from mtcnn.mtcnn import MTCNN
import numpy as np
detector1 = MTCNN()
modelFile = "res10_300x300_ssd_iter_140000.caffemodel"
configFile = "deploy.prototxt.txt"
net = cv2.dnn.readNetFromCaffe(configFile, modelFile)

cap = cv2.VideoCapture('video.mp4') #window with video from camera
font = cv2.FONT_ITALIC #denotes the font type of text in video
line = cv2.FILLED  #linetype
while(True):
    ret, img = cap.read()
    if ret == True:
        img = cv2.resize(img, None, fx=1, fy=1)
        height, width = img.shape[:2]
        img1 = img.copy()
        img2 = img.copy()
        img3 = img.copy()
        
        blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)),
                                        1.0, (300, 300), (104.0, 117.0, 123.0))
        net.setInput(blob)
        faces1 = detector1.detect_faces(img)# result for mtcnn
        faces2 = net.forward() # result for dnn
        
    # display faces on the real time     
        #MTCNN
        for result in faces1:
            x, y, w, h = result['box']
            x1, y1 = x + w, y + h
            #draw a rectangle (image, start_point, end_point, color, thickness):
            cv2.rectangle(img, (x, y), (x1, y1), (0, 0, 255), 2)
        #put text into video
        #(image, text, org, font, fontScale, color[, thickness[, lineType[, bottomLeftOrigin]]]
        cv2.putText(img, 'MTCNN', (30, 30), font, 1, (255, 255, 0), 2, line)

        #dnn
        for i in range(faces2.shape[2]):
            confidence = faces2[0, 0, i, 2]
            if confidence > 0.5: 
                box = faces2[0, 0, i, 3:7] * np.array([width, height, width, height])
                (x, y, x1, y1) = box.astype("int")
                #draw a rectangle (image, start_point, end_point, color, thickness):
                cv2.rectangle(img2, (x, y), (x1, y1), (0, 0, 255), 2)
        #put text into video 
        cv2.putText(img2, 'DNN', (30, 30), font, 1, (255, 255, 0), 2, line)
        
        h1 = cv2.hconcat([img3, img1])
        h2 = cv2.hconcat([img, img2])
        fin = cv2.vconcat([h1, h2])
        
        cv2.imshow("MTCNN", img) #present the result
        cv2.imshow("DNN", img2) #present the result
        if cv2.waitKey(1) & 0xFF == ord('q'): #press "q" to out
            break
    else:
        break


           
cap.release()
cv2.destroyAllWindows()

Подивимось на швидкість обробки відео:

In [2]:
%%time
net = cv2.dnn.readNetFromCaffe("models/deploy.prototxt.txt",
                               "models/res10_300x300_ssd_iter_140000.caffemodel")

cap = cv2.VideoCapture('video.mp4') #window with video from camera
font = cv2.FONT_ITALIC #denotes the font type of text in video
line = cv2.FILLED  #linetype
while(True):
    ret, img = cap.read()
    if ret == True:
        img = cv2.resize(img, None, fx=1, fy=1)
        height, width = img.shape[:2]
        img1 = img.copy()
        img2 = img.copy()
        img3 = img.copy()
        
        blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)),
                                        1.0, (300, 300), (104.0, 117.0, 123.0))
        net.setInput(blob)
        faces2 = net.forward() 
        
         #put text into video
                #dnn
        for i in range(faces2.shape[2]):
            confidence = faces2[0, 0, i, 2]
            if confidence > 0.5: 
                box = faces2[0, 0, i, 3:7] * np.array([width, height, width, height])
                (x, y, x1, y1) = box.astype("int")
                #draw a rectangle (image, start_point, end_point, color, thickness):
                cv2.rectangle(img2, (x, y), (x1, y1), (0, 0, 255), 2)
        #put text into video 
        cv2.putText(img2,
                    'DNN', (30, 30),
                    font, 1, (255, 255, 0), 2, line)
        
        h1 = cv2.hconcat([img3, img1])
        h2 = cv2.hconcat([img, img2])
        fin = cv2.vconcat([h1, h2])
        
        cv2.imshow("DNN", img2) #present the result
        if cv2.waitKey(1) & 0xFF == ord('q'): #press "q" to out
            break
    else:
        break


           
cap.release()
cv2.destroyAllWindows()

Wall time: 11.3 s


In [3]:
%%time
detector1 = MTCNN()

cap = cv2.VideoCapture('video.mp4') #window with video from camera
font = cv2.FONT_ITALIC #denotes the font type of text in video
line = cv2.FILLED  #linetype
while(True):
    ret, img = cap.read()
    if ret == True:
        img = cv2.resize(img, None, fx=1, fy=1)
        height, width = img.shape[:2]
        img1 = img.copy()
        img2 = img.copy()
        img3 = img.copy()
        
        blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)),
                                        1.0, (300, 300), (104.0, 117.0, 123.0))
        net.setInput(blob)
        faces1 = detector1.detect_faces(img)# result for mtcnn
        faces2 = net.forward() # result for dnn
        
    # display faces on the real time     
        #MTCNN
        for result in faces1:
            x, y, w, h = result['box']
            x1, y1 = x + w, y + h
            #draw a rectangle (image, start_point, end_point, color, thickness):
            cv2.rectangle(img, (x, y), (x1, y1), (0, 0, 255), 2)
        #put text into video
        
        cv2.putText(img, 'MTCNN', (30, 30), font, 1, (255, 255, 0), 2, line)

        h1 = cv2.hconcat([img3, img1])
        h2 = cv2.hconcat([img, img2])
        fin = cv2.vconcat([h1, h2])
        
        cv2.imshow("MTCNN", img) #present the result
        if cv2.waitKey(1) & 0xFF == ord('q'): #press "q" to out
            break
    else:
        break

           
cap.release()
cv2.destroyAllWindows()

Wall time: 24.7 s


При обробці завантаженого відео DNN знадобилось 13.9 s, а MTCNN - 1min 23s. Працює набагато швидше DNN.
При тестуванні з камери  DNN  модель краще показала себе з оклюзією, швидкими рухами голови. Змогла ідентифікувати бічні грані. Гарно справилась з перепадами освітлення та перешкодами(частково закритим лицем). 
Висновок - використовуємо модель Caffe модуля DNN від OpenCV.

In [4]:
net = cv2.dnn.readNetFromCaffe("deploy.prototxt.txt",
                               "res10_300x300_ssd_iter_140000.caffemodel")

cap = cv2.VideoCapture(0) #window with video from camera
font = cv2.FONT_ITALIC #denotes the font type of text in video
line = cv2.FILLED  #linetype
while(True):
    ret, img = cap.read()
    if ret == True:
        img = cv2.resize(img, None, fx=1, fy=1)
        height, width = img.shape[:2]
        img1 = img.copy()
        img2 = img.copy()
        img3 = img.copy()
        
        blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)),
                                        1.0, (300, 300), (104.0, 117.0, 123.0))
        net.setInput(blob)
        faces2 = net.forward() 
        
         #put text into video
                #dnn
        for i in range(faces2.shape[2]):
            confidence = faces2[0, 0, i, 2]
            if confidence > 0.5: 
                box = faces2[0, 0, i, 3:7] * np.array([width, height, width, height])
                (x, y, x1, y1) = box.astype("int")
                #draw a rectangle (image, start_point, end_point, color, thickness):
                cv2.rectangle(img2, (x, y), (x1, y1), (0, 0, 255), 2)
        #put text into video 
        cv2.putText(img2,
                    'DNN', (30, 30),
                    font, 1, (255, 255, 0), 2, line)
        
        h1 = cv2.hconcat([img3, img1])
        h2 = cv2.hconcat([img, img2])
        fin = cv2.vconcat([h1, h2])
        
        cv2.imshow("DNN", img2) #present the result
        if cv2.waitKey(1) & 0xFF == ord('q'): #press "q" to out
            break
    else:
        break


           
cap.release()
cv2.destroyAllWindows()